In [1]:
import csv
import nltk
import pandas as pd

In [2]:
pd.set_option('display.max_colwidth', -1)

In [3]:
df = pd.read_csv("WITBrag_Clean_Noretweets.csv")

In [4]:
from nltk.tokenize import word_tokenize
import re

emoticons_str = r"""
    (?:
        [:=;] # Eyes
        [oO\-]? # Nose (optional)
        [D\)\]\(\]/\\OpP] # Mouth
    )"""
 
regex_str = [
    emoticons_str,
    r'<[^>]+>', # HTML tags
    r'(?:@[\w_]+)', # @-mentions
    r"(?:\#+[\w_]+[\w\'_\-]*[\w_]+)", # hash-tags
    r'http[s]?://(?:[a-z]|[0-9]|[$-_@.&amp;+]|[!*\(\),]|(?:%[0-9a-f][0-9a-f]))+', # URLs
 
    r'(?:(?:\d+,?)+(?:\.?\d+)?)', # numbers
    r"(?:[a-z][a-z'\-_]+[a-z])", # words with - and '
    r'(?:[\w_]+)', # other words
    r'(?:\S)' # anything else
]
    
tokens_re = re.compile(r'('+'|'.join(regex_str)+')', re.VERBOSE | re.IGNORECASE)
emoticon_re = re.compile(r'^'+emoticons_str+'$', re.VERBOSE | re.IGNORECASE)

def remove_emoji(data):
    if not data:
        return data
    if not isinstance(data, basestring):
        return data
    try:
    # UCS-4
        patt = re.compile(u'([\U00002600-\U000027BF])|([\U0001f300-\U0001f64F])|([\U0001f680-\U0001f6FF])')
    except re.error:
    # UCS-2
        patt = re.compile(u'([\u2600-\u27BF])|([\uD83C][\uDF00-\uDFFF])|([\uD83D][\uDC00-\uDE4F])|([\uD83D][\uDE80-\uDEFF])')
    return patt.sub('', data)
 
def tokenize(s):
    return tokens_re.findall(s)
 
def preprocess(s, lowercase=False):
    s = s.decode('unicode-escape').encode('latin1').decode('utf8')
    tokens = tokenize(s)
    if lowercase:
        tokens = [token if emoticon_re.search(token) else token.lower() for token in tokens]
    return tokens

test_tweet = 'RT @alicegoldfuss: So happy to hear women getting recruiter pings after #WITBragDay :D'
print (preprocess(test_tweet))

[u'RT', u'@alicegoldfuss', u':', u'So', u'happy', u'to', u'hear', u'women', u'getting', u'recruiter', u'pings', u'after', u'#WITBragDay', u':D']


In [5]:
test = df.head(20).text

In [6]:
test

0     3 degrees in building industry and they rejected me because #women. IT industry welcomed me with open arms. Not a single regret #WITBragDay          
1     @azimman @LaunchDarkly sounds like you should look at the #WITBragDay tag :P                                                                         
2     My coding knowledge makes a difference and improves the lives of my coworkers and our customers. And it's the coolest feeling. #WITBragDay           
3     Built a ECS JSON-generating container definition module entirely in Terraform, because why would you not. #WITBragDay                                
4     Recruiters should definitely be scrolling through #WITBragDay for their lists. Tons of awesome talent in there.                                      
5     Started a company doing DevOps consulting. Still proud of this one. 😄 #WITBragDay                                                                   
6     I just learned about #WITBragDay and wish I would've partic

In [7]:
test[7]

'\xf0\x9f\x91\x8d\xf0\x9f\x8f\xbb\nRT @matthewbretten: @ColetteWeston @drjessicabarker Definitely a #WITBragDay'

In [8]:
preprocess(test[7])

[u'\ud83d',
 u'\udc4d',
 u'\ud83c',
 u'\udffb',
 u'RT',
 u'@matthewbretten',
 u':',
 u'@ColetteWeston',
 u'@drjessicabarker',
 u'Definitely',
 u'a',
 u'#WITBragDay']

In [11]:
import unicodedata as ud
test = test[7].decode('unicode-escape').encode('latin1').decode('utf8')

In [12]:
test

u'\U0001f44d\U0001f3fb\nRT @matthewbretten: @ColetteWeston @drjessicabarker Definitely a #WITBragDay'

In [16]:
preprocess(test)

UnicodeEncodeError: 'ascii' codec can't encode characters in position 0-3: ordinal not in range(128)

In [17]:
def remove_emoji(data):
    if not data:
        return data
    if not isinstance(data, basestring):
        return data
    try:
    # UCS-4
        patt = re.compile(u'([\U00002600-\U000027BF])|([\U0001f300-\U0001f64F])|([\U0001f680-\U0001f6FF])')
    except re.error:
    # UCS-2
        patt = re.compile(u'([\u2600-\u27BF])|([\uD83C][\uDF00-\uDFFF])|([\uD83D][\uDC00-\uDE4F])|([\uD83D][\uDE80-\uDEFF])')
    return patt.sub('', data)

In [18]:
remove_emoji(test)

u'\nRT @matthewbretten: @ColetteWeston @drjessicabarker Definitely a #WITBragDay'